In [2]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [3]:
def mygraph(mnist):
    x = tf.placeholder('float',[None, 784])
    w = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros(10))
    y = tf.nn.softmax(tf.matmul(x,w)+b)
    y_true = tf.placeholder('float', [None, 10])
    cross_entropy = -tf.reduce_sum(y_true*tf.log(y))
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    for _ in range(10000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step,feed_dict={x: batch_xs, y_true: batch_ys})
    correct_prediction = tf.equal(tf.arg_max(y, 1), tf.argmax(y_true, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_true: mnist.test.labels}))

def graph_v3(mnist):
    sess = tf.InteractiveSession()
    x = tf.placeholder('float', shape=[None, 784])
    y_true = tf.placeholder('float', shape=[None, 10])
    w = tf.Variable(tf.zeros([784,10]))
    b = tf.Variable(tf.zeros([10]))
    sess.run(tf.initialize_all_variables())
    y = tf.nn.softmax(tf.matmul(x,w) + b)
    cross_entropy = -tf.reduce_sum(y_true*tf.log(y))
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
    for _ in range(1000):
        batch = mnist.train.next_batch(100)
        train_step.run(feed_dict={x:batch[0], y_true:batch[1]})
    correct_prediction = tf.equal(tf.arg_max(y,1), tf.arg_max(y_true,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    print(accuracy.eval(feed_dict={x:mnist.test.images, y_true:mnist.test.labels}))

In [4]:
mnist = input_data.read_data_sets('data/', one_hot=True)
#graph_v3(mnist)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

def my_conv_graph(mnist):
    sess = tf.InteractiveSession()
    x = tf.placeholder('float', shape=[None, 784])
    y_true = tf.placeholder('float', shape=[None, 10])
    x_image = tf.reshape(x, [-1,28,28,1])
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    keep_prob = tf.placeholder("float")
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    cross_entropy = -tf.reduce_sum(y_true*tf.log(y_conv))
    
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_true,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    sess.run(tf.initialize_all_variables())
    for i in range(2000):
        batch = mnist.train.next_batch(50)
        if i%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_true: batch[1], keep_prob: 1.0})
            print("step %d, training accuracy %g"%(i, train_accuracy))
            print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_true: mnist.test.labels, keep_prob: 1.0}))
        train_step.run(feed_dict={x: batch[0], y_true: batch[1], keep_prob: 0.5})

In [7]:
my_conv_graph(mnist)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, training accuracy 0.22
test accuracy 0.1086
step 100, training accuracy 0.88
test accuracy 0.8455
step 200, training accuracy 0.94
test accuracy 0.8995
step 300, training accuracy 0.94
test accuracy 0.9269
step 400, training accuracy 0.88
test accuracy 0.9359
step 500, training accuracy 0.88
test accuracy 0.9408
step 600, training accuracy 0.98
test accuracy 0.9486
step 700, training accuracy 0.94
test accuracy 0.9535
step 800, training accuracy 0.98
test accuracy 0.9603
step 900, training accuracy 0.92
test accuracy 0.9607
step 1000, training accuracy 0.92
test accuracy 0.9625
step 1100, training accuracy 0.98
test accuracy 0.9652
step 1200, training accuracy 0.96
test accuracy 0.9673
step 1300, training accuracy 0.98
te